In [12]:
import numpy as np
from numpy import nan
import matplotlib.pyplot as plt

from tabulate import tabulate
import random
from itertools import zip_longest
from collections import namedtuple
from stack import Stack
from actions import Moove, Action, List_actions
from utils_class import EpsilonGreedy, ReplayMemory
from DQN import DQNAgent
from env import Env 
from constant import *

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
size = 10
env = Env(size)
# print(env.state().shape[0])
agent = DQNAgent(size, len(env.action_space()))

In [14]:
episode_durations = list()
for episode in range(EPISODE):
    
    env.reset(STACK_SIZE)
    state = env.state()
    # env.print_state()
    
    # while not (env.state == 'done'):
    timesteps = 0
    for i in range(2):
        
        timesteps += 1;
        action = env.choose_action(agent.policy_model(state))
        reward = env.reward()
        next_state = env.state()
        
        experience = env.create_experience(state, action, next_state, reward)
        env.replaymemory.push(experience)
        # env.print_experience(experience)
        state = next_state

        if env.replaymemory.can_provide_sample(BATCH_SIZE):
            
            experiences = env.replaymemory.get_sample(BATCH_SIZE)
            states, actions, next_states, rewards = env.replaymemory.extract_value(experiences)
            # env.replaymemory.print_buffer(experiences)
            agent.train(states, next_states, experiences)
        

    if (env.state == 'done'):
        episode_durations.append(timesteps)
plt.plot(episode_durations)    
            
    
    

STATESSSSS [[[97. 39. 74. 54. 35. 53. 37. 27. 77. 18.]
  [nan nan nan nan nan nan nan nan nan nan]]

 [[39. 74. 54. 35. 53. 37. 27. 77. 18. 97.]
  [nan nan nan nan nan nan nan nan nan nan]]]
(2, 2, 10)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_203447/527718494.py", line 27, in <cell line: 2>
    agent.train(states, next_states, experiences)
  File "/home/widium/Programming/AI/RL_push_swap/DQN.py", line 39, in train
    Q_policy_list = self.policy_model.predict(states)
  File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/tensorflow/python/framework/func_graph.py", line 1129, in autograph_handler
    raise e.ag_error_metadata.to_exception(e)
ValueError: in user code:

    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        

In [11]:
# env.print_stack()
# env.moover.push_b()
# env.choose_action()
env.current_action = 0
env.take_actions()
env.create_experience()
env.replaymemory.push(env.experience)
env.current_action = 1
env.take_actions()
env.create_experience()
env.replaymemory.push(env.experience)
env.current_action = 0
env.take_actions()
env.create_experience()
env.replaymemory.push(env.experience)
env.take_actions()
env.create_experience()
env.replaymemory.push(env.experience)
# print(env.experience)
env.print_state()


---State t---
-A-	-B-
___________	
|52.0	4.0|
|15.0	92.0|
|48.0	nan|
|68.0	nan|
|8.0	nan|
|89.0	nan|
|78.0	nan|
|60.0	nan|
|nan	nan|
|nan	nan|
___________	


In [6]:
env.replaymemory.get_sample(4)

[Experience(state=array([[nan, 13., 27., 84., 58., 93., 67., 35., 11., 71.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, 26.]], dtype=float32), action=1, next_state=array([[13., 27., 84., 58., 93., 67., 35., 11., 71., 26.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32), reward=-6),
 Experience(state=array([[13., 27., 84., 58., 93., 67., 35., 11., 71., 26.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32), action=0, next_state=array([[nan, 13., 27., 84., 58., 93., 67., 35., 11., 71.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, 26.]], dtype=float32), reward=-1),
 Experience(state=array([[13., 27., 84., 58., 93., 67., 35., 11., 71., 26.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32), action=0, next_state=array([[nan, 13., 27., 84., 58., 93., 67., 35., 11., 71.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, 26.]], dtype=float32), reward=-6),
 Experience(state=array([[nan, 

In [7]:
env.replaymemory.print_history(BATCH_SIZE)

Actions : 0 	 Reward : -1
Actions : 1 	 Reward : -6
Actions : 0 	 Reward : -6
Actions : 0 	 Reward : -1
